In [1]:
%%time
%reload_ext autoreload
%autoreload
%autoreload 2
%config Completer.use_jedi = False

import os
import json
import warnings
import pprint
import sys
import numpy as np
import pandas as pd
import torch

MAIN_PATH = os.getcwd().split("notebooks")[0]
sys.path.insert(0, MAIN_PATH)

CPU times: user 2.24 s, sys: 848 ms, total: 3.09 s
Wall time: 3.18 s


In [2]:
# Run a Federated Learning experiment
from data_loader.mnist import MnistDatasetManager
from models.mnist_model import MnistFullConnectModel, SimpleCNN
from server.base_server import BaseServer
from client.base_client import BaseClient
from experiments.base_experiment import BaseExperiment
from gradients.noise import GaussianNoiseGenerator, NoNoiseGenerator,StaircaseNoiseGenerator
from metrics.classification import multiclass_accuracy

In [3]:
class DemoMnistExperiment(BaseExperiment):
    def __init__(self, client_num: int = 2, lr: float=0.01):
        self.lr = lr
        self._init_server_clients(client_num,self.lr)
        self._init_data(client_num)

    def _init_server_clients(self, client_num, lr):
        model = SimpleCNN
        self.clients = [BaseClient(model(lr=lr), 
                                   client_id=idx, 
                                   noise_generator=StaircaseNoiseGenerator())
                        for idx in range(client_num)]
        self.server = BaseServer(model(lr=lr))

    def _init_data(self, client_num):
        data_manager = MnistDatasetManager(n_parties=client_num)
        self.client_train_datas = data_manager.train_loaders
        self.valid_datas = data_manager.validation_loader
        self.test_data = data_manager.test_loader

    def evaluate_model(self, data):
        total_correct = 0
        total_sample_num = 0
        with torch.no_grad():
            for _, (inputs, target) in enumerate(data):
                predict_labels = self.server.predict(inputs)
                correct, sample_num = multiclass_accuracy(y_pred=predict_labels, 
                                                          y_true=target)
                total_correct += correct
                total_sample_num += sample_num
                
        return total_correct / total_sample_num
        
    def get_validation_result(self):
        return self.evaluate_model(self.valid_datas)
    
    def get_test_result(self):
        return self.evaluate_model(self.test_data)
    
    def aggeragate(self):
        self.server.aggeragate_model(self.clients)
    
    def run(self, epochs: int):
        for client in self.clients:
            client.set_training_mode(for_gradient=False)

        for epoch in range(epochs):
            print(self.get_validation_result())
            for client, client_train_data in self.shuffled_data(to_shuffle=False):
                client.train(client_train_data, client_epochs=2)

            self.aggeragate()
            if epoch and not (epoch % 1):
                print(self.get_validation_result())

            self.distribute_model()


In [4]:
EXPERIMENT = DemoMnistExperiment(client_num=10, lr = 0.001)

In [5]:
EXPERIMENT.run(50)

0.11191666666666666
0.21358333333333332
0.94325
0.94325
0.9695
0.9695
0.9761666666666666
0.9761666666666666
0.9795
0.9795
0.98325
0.98325
0.9845
0.9845
0.9864166666666667
0.9864166666666667
0.98725
0.98725
0.987
0.987
0.9885833333333334
0.9885833333333334
0.98875
0.98875
0.9893333333333333
0.9893333333333333
0.9899166666666667
0.9899166666666667
0.9901666666666666
0.9901666666666666
0.9900833333333333
0.9900833333333333
0.9903333333333333
0.9903333333333333
0.9906666666666667
0.9906666666666667
0.991
0.991
0.9906666666666667
0.9906666666666667
0.9909166666666667
0.9909166666666667
0.9915
0.9915
0.9915
0.9915
0.9915833333333334
0.9915833333333334
0.9918333333333333
0.9918333333333333
0.9920833333333333
0.9920833333333333
0.9923333333333333
0.9923333333333333
0.99225
0.99225
0.9921666666666666
0.9921666666666666
0.99175
0.99175
0.9924166666666666
0.9924166666666666
0.9921666666666666
0.9921666666666666
0.992
0.992
0.9923333333333333
0.9923333333333333
0.99225
0.99225
0.9923333333333333
0

In [6]:
EXPERIMENT.clients[0].train

<bound method BaseClient.train_for_model of <client.base_client.BaseClient object at 0x7fa633d4eb50>>